# Concert enjoyment | Best classifier
This notebook includes only the best classifier (XGBoost) that was developed by the team 'The Overfitters' in the Kaggle competition.

INF8245E, December 2022

## Instructions on how to run code





1.   Under section 0. *Loading data and packages*, in cell [2], change the path of the test set by changing the path parameter in the following function call
`test_data = pd.read_csv("./public/data/test.csv")` from `"./public/data/test.csv"` to the path of the .csv file that contains the test set
2.   Run the entire notebook by going to *Run* -> *Run all*
3.   A file called output.csv, containing the output predictions on the test set, will be downloaded automatically

## 0. Loading data and packages

In [1]:
!rm -rf public
!git clone https://github.com/zhanwen-xin/concert-enjoyment-classification-data.git public

Cloning into 'public'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 37 (delta 7), reused 23 (delta 2), pack-reused 0
Unpacking objects: 100% (37/37), done.


In [2]:
# Import Packages  
import pandas as pd
import numpy as np
import sklearn

# Load data
train_data = pd.read_csv("./public/data/train.csv")
test_data = pd.read_csv("./public/data/test.csv") #Change this path to the correct .csv file that contains the test set

### 0.1 Visualize dataset

#### Original training set

In [3]:
train_data

,Id,Band Name,Band Genre,Band Country of Origin,Band Debut,Concert ID,Concert Attendance,Inside Venue,Rain,Seated,Personnality Trait 1,Personnality Trait 2,Personnality Trait 3,Personnality Trait 4,Concert Goer Age,Concert Goer ID,Height (cm),Concert Goer Country of Origin,Concert Enjoyment
0,ConcertExperience_180106,Teenage Crazy Blue Knickers,Indie/Alt Rock,United States of America (USA),1976.0,900.0,2980.0,False,False,NaN,0.330843,-0.958408,-0.943548,-1.636806,29.0,concert_goer_1985,140.0,Paraguay,Did Not Enjoy
1,ConcertExperience_146268,Beyond Devon,Pop Music,United States of America (USA),1968.0,731.0,54.0,True,False,True,-2.069449,0.017777,-1.910675,0.610265,43.0,concert_goer_1874,158.0,United Kingdom (UK),Enjoyed
2,ConcertExperience_128743,Ron Talent,Rock n Roll,Canada,1955.0,NaN,162754.0,False,False,True,-0.484268,1.968772,-0.064167,-1.260871,68.0,concert_goer_442,159.0,United States of America (USA),Did Not Enjoy
3,ConcertExperience_140839,Devon Revival,RnB,United States of America (USA),1992.0,704.0,8103.0,False,True,False,-0.858054,1.022827,-0.348389,-1.147251,17.0,concert_goer_1149,150.0,Canada,Worst Concert Ever
4,ConcertExperience_19149,Beyond Devon,Pop Music,United States of America (USA),1968.0,95.0,54.0,False,False,False,-0.793029,-1.166528,-0.043766,0.969661,59.0,concert_goer_930,166.0,United Kingdom (UK),Did Not Enjoy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169995,ConcertExperience_14055,Crazy Joystick Cult,RnB,Canada,1985.0,70.0,162754.0,True,False,False,-0.095021,0.175175,0.914245,0.357359,50.0,concert_goer_707,180.0,United States of America (USA),Did Not Enjoy
169996,ConcertExperience_192792,Crazy Joystick Cult,RnB,Canada,1985.0,963.0,54.0,False,False,False,-0.733719,-0.285776,-0.323312,0.641180,71.0,concert_goer_1373,143.0,Bulgaria,Worst Concert Ever
169997,ConcertExperience_152942,"Why Frogs, Why?",Heavy Metal,Canada,2005.0,764.0,54.0,False,False,False,0.744969,-0.965547,1.020598,1.027389,27.0,concert_goer_1286,176.0,Canada,Did Not Enjoy
169998,ConcertExperience_138957,Twilight of the Joystick Gods,Hip Hop/Rap,United States of America (USA),1995.0,694.0,22026.0,False,True,True,0.821976,0.351411,0.175762,1.455654,39.0,concert_goer_1845,176.0,Canada,Did Not Enjoy


#### Original test set: only 30% of the test set accessible

In [4]:
test_data

,Id,Band Name,Band Genre,Band Country of Origin,Band Debut,Concert ID,Concert Attendance,Inside Venue,Rain,Seated,Personnality Trait 1,Personnality Trait 2,Personnality Trait 3,Personnality Trait 4,Concert Goer Age,Concert Goer ID,Height (cm),Concert Goer Country of Origin
0,ConcertExperience_70055,The Crazy Heroes of Devon,Rock n Roll,United States of America (USA),1980.0,350.0,2980.0,True,False,True,1.065107,0.057660,0.249639,-0.933976,74.0,concert_goer_1587,165.0,United States of America (USA)
1,ConcertExperience_34799,Joystick for the Jockies,Hip Hop/Rap,United States of America (USA),2014.0,173.0,8103.0,True,True,False,-0.886947,0.801365,0.525624,0.176655,29.0,concert_goer_293,151.0,Kenya
2,ConcertExperience_100410,Puddle of Joystick,Rock n Roll,Canada,2010.0,502.0,2980.0,True,True,False,0.744700,-0.797531,-0.034166,-0.226052,27.0,concert_goer_1068,146.0,Canada
3,ConcertExperience_106446,Flight of the Knickers,NaN,Canada,2014.0,532.0,22026.0,True,False,False,-0.134180,-0.361512,0.969404,-2.341205,38.0,concert_goer_1315,183.0,United States of America (USA)
4,ConcertExperience_127249,Devon Revival,RnB,United States of America (USA),1992.0,636.0,2980.0,False,False,False,1.407366,-0.084155,-0.673233,1.733714,21.0,concert_goer_1777,177.0,Fiji
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,ConcertExperience_82288,Joystick of the Big Knickers,Hip Hop/Rap,United States of America (USA),1979.0,411.0,2980.0,True,False,False,-0.421714,-1.549670,-0.351770,0.132489,42.0,concert_goer_1710,178.0,United States of America (USA)
29996,ConcertExperience_27139,Big Division,Hip Hop/Rap,United States of America (USA),1978.0,135.0,8103.0,True,False,False,0.615087,-0.047092,0.339228,0.820159,37.0,concert_goer_1758,186.0,Canada
29997,ConcertExperience_197434,Crazyplay,Pop Music,United States of America (USA),1995.0,987.0,8103.0,False,False,True,-1.396551,-0.508627,-1.692584,1.640931,45.0,concert_goer_1481,158.0,Greece
29998,ConcertExperience_166029,Lord of the Crazy Frogs,RnB,United States of America (USA),1968.0,830.0,8103.0,False,True,False,0.168073,-0.785460,0.898273,1.608389,36.0,concert_goer_1461,170.0,United Kingdom (UK)


## 1. Functions for data preprocessing

### 1.1 Filter NaN with mean

In [12]:
def nan_replace_with_mean(data):
  '''
  This function is to transform the boolean value True to float 1, False to float 0.0 and NaN to 0.5
  args: 
    Input:
        data(pandas.core.frame.DataFrame): one column of data to fit the encoder [shape: (# dataset,1)]

    Return:
        transformed_data(pandas.core.frame.DataFrame): data after transformation by the fit encoder [shape: (# dataset,1)]
  '''
  mean=data.mean(axis=0)
  transformed_data=data.fillna(mean)
  return transformed_data

### 1.2 One Hot encoder

In [13]:
def onehot_encoder_fit_transform(train_data,test_data,column_head='none'):
  '''
  args: 
    Input:
        train_data(pandas.core.frame.DataFrame): one column of data to fit the encoder [shape: (#fitset,1)]
        test_data(pandas.core.frame.DataFrame): one column of data to transform by the encoder [shape: (#dataset to transform,1)]
        column_head(string): name of column that user wishes to define

    Return:
        transformed_data(pandas.core.frame.DataFrame): data after transformation by the fit encoder [shape: (#dataset transformed,#categories in fitset)]
  '''
  from sklearn import preprocessing
  encoder = preprocessing.OneHotEncoder(handle_unknown='ignore')
  encoder.fit(train_data)
  transformed_data=pd.DataFrame(encoder.transform(test_data).toarray())

  col_num=len(transformed_data.columns)
  column_name=[]
  for i in range(col_num):
    column_name.append('%s%d' % (column_head, i))

  transformed_data.columns = column_name
  return transformed_data

### 1.3 Label binarizer encoder

In [14]:
def label_binarizer_transform(data):
  '''
  This function is to transform the boolean value True to float 1, False to float 0.0 and NaN to 0.5
  args: 
    Input:
        data(pandas.core.frame.DataFrame): one column of data to fit the encoder [shape: (# dataset to transform,1)]

    Return:
        transformed_data(pandas.core.frame.DataFrame): data after transformation by the fit encoder [shape: (# dataset to transform,1)]
  '''
  transformed_data=data*1
  transformed_data=transformed_data.fillna(0.5)
  return transformed_data

### 1.4 MinMax scalar

In [15]:
def minmax_scalar(data):
  from sklearn.preprocessing import MinMaxScaler
  scaler_mm = MinMaxScaler()
  # nomalize data with minimum and maximum value
  data = scaler_mm.fit_transform(data)
  return data

### 1.5 Standardization scalar

In [16]:
def standardize_scalar(data):
  from sklearn.preprocessing import StandardScaler
  scaler_ss = StandardScaler()
  # standardize data
  data = scaler_ss.fit_transform(data)
  return data

### 1.6 Shuffle

In [17]:
def shuffle(y, X, seed=0):
    obs = X.shape[0]
    rng = np.random.default_rng(seed)
    perm = rng.permutation(obs)
    y = y[perm]
    X = X[perm, :]
    return (y, X)

### 1.7 Imputer

In [18]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import BayesianRidge

def imputeDataEstimator(data, estimator=BayesianRidge()):
  """Performs multivariate data imputation on NaN values using the given estimator

  Args:
    data (pandas.DataFrame): DataFrame of the entire dataset with NaNs
    estimator (estimator model): Model used to estimate NaN values

  Returns:
    data (pandas.DataFrame): DataFrame of the entire dataset without NaNs
  """
  imp = IterativeImputer(estimator=estimator, max_iter=10, random_state=0)
  data = imp.fit_transform(data.to_numpy())
  data = pd.DataFrame(data)
  return data

## 2. Data preprocessing

### 2.1 Preprocess training and test attributes

In [19]:
# Band Name column
train_column=train_data[['Band Name']]
BNtrain_transformed=onehot_encoder_fit_transform(train_column,train_column,column_head='BN').to_numpy()
test_column=test_data[['Band Name']]
BNtest_transformed=onehot_encoder_fit_transform(train_column,test_column,column_head='BN').to_numpy()

In [20]:
# Band Genre column
train_column=train_data[['Band Genre']]
BGtrain_transformed=onehot_encoder_fit_transform(train_column,train_column,column_head='BG').to_numpy()
test_column=test_data[['Band Genre']]
BGtest_transformed=onehot_encoder_fit_transform(train_column,test_column,column_head='BG').to_numpy()

In [21]:
# Band Country of Origin column
train_column=train_data[['Band Country of Origin']]
BCtrain_transformed=onehot_encoder_fit_transform(train_column,train_column,column_head='BC').to_numpy()
test_column=test_data[['Band Country of Origin']]
BCtest_transformed=onehot_encoder_fit_transform(train_column,test_column,column_head='BC').to_numpy()
#print(BCcolumn_transformed)

In [22]:
# Band Debut column
train_column=train_data[['Band Debut']]
BDtrain_transformed=nan_replace_with_mean(train_column).to_numpy()
test_column=test_data[['Band Debut']]
BDtest_transformed=nan_replace_with_mean(test_column).to_numpy()

In [23]:
# Concert ID column
train_column=train_data[['Concert ID']]
CItrain_transformed=nan_replace_with_mean(train_column).to_numpy()
test_column=test_data[['Concert ID']]
CItest_transformed=nan_replace_with_mean(test_column).to_numpy()

In [24]:
# Concert Attendance column
train_column=train_data[['Concert Attendance']]
CAtrain_transformed=nan_replace_with_mean(train_column).to_numpy()
test_column=test_data[['Concert Attendance']]
CAtest_transformed=nan_replace_with_mean(test_column).to_numpy()

In [25]:
# Personnality Trait 1 column
train_column=train_data[['Personnality Trait 1']]
PT1train_transformed=nan_replace_with_mean(train_column).to_numpy()
test_column=test_data[['Personnality Trait 1']]
PT1test_transformed=nan_replace_with_mean(test_column).to_numpy()

In [26]:
# Personnality Trait 2 column
train_column=train_data[['Personnality Trait 2']]
PT2train_transformed=nan_replace_with_mean(train_column).to_numpy()
test_column=test_data[['Personnality Trait 2']]
PT2test_transformed=nan_replace_with_mean(test_column).to_numpy()

In [27]:
# Personnality Trait 3 column
train_column=train_data[['Personnality Trait 3']]
PT3train_transformed=nan_replace_with_mean(train_column).to_numpy()
test_column=test_data[['Personnality Trait 3']]
PT3test_transformed=nan_replace_with_mean(test_column).to_numpy()

In [28]:
# Personnality Trait 4 column
train_column=train_data[['Personnality Trait 4']]
PT4train_transformed=nan_replace_with_mean(train_column).to_numpy()
test_column=test_data[['Personnality Trait 4']]
PT4test_transformed=nan_replace_with_mean(test_column).to_numpy()

In [29]:
# Concert Goer Age column
train_column=train_data[['Concert Goer Age']]
CGAtrain_transformed=nan_replace_with_mean(train_column).to_numpy()
test_column=test_data[['Concert Goer Age']]
CGAtest_transformed=nan_replace_with_mean(test_column).to_numpy()

In [30]:
# Height (cm) column
train_column=train_data[['Height (cm)']]
HTtrain_transformed=nan_replace_with_mean(train_column).to_numpy()
test_column=test_data[['Height (cm)']]
HTtest_transformed=nan_replace_with_mean(test_column).to_numpy()

In [31]:
# Inside Venue column
train_column=train_data[['Inside Venue']]
IVtrain_transformed=label_binarizer_transform(train_column).to_numpy()
test_column=test_data[['Inside Venue']]
IVtest_transformed=label_binarizer_transform(test_column).to_numpy()
#print(IVcolumn_transformed)

In [32]:
# Rain column
train_column=train_data[['Rain']]
RNtrain_transformed=label_binarizer_transform(train_column).to_numpy()
test_column=test_data[['Rain']]
RNtest_transformed=label_binarizer_transform(test_column).to_numpy()
#print(RNcolumn_transformed)

In [33]:
# Seated column
train_column=train_data[['Seated']]
STtrain_transformed=label_binarizer_transform(train_column).to_numpy()
test_column=test_data[['Seated']]
STtest_transformed=label_binarizer_transform(test_column).to_numpy()
#print(STcolumn_transformed)

In [34]:
# Concert Goer ID column
train_column=train_data[['Concert Goer ID']]
CGtrain_transformed=onehot_encoder_fit_transform(train_column,train_column,column_head='CG').to_numpy()
test_column=test_data[['Concert Goer ID']]
CGtest_transformed=onehot_encoder_fit_transform(train_column,test_column,column_head='CG').to_numpy()
#print(CGcolumn_transformed)

In [35]:
# Concert Goer Country of Origin column
train_column=train_data[['Concert Goer Country of Origin']]
CGCtrain_transformed=onehot_encoder_fit_transform(train_column,train_column,column_head='CGC').to_numpy()
test_column=test_data[['Concert Goer Country of Origin']]
CGCtest_transformed=onehot_encoder_fit_transform(train_column,test_column,column_head='CGC').to_numpy()
#print(CGCtrain_transformed.shape)

### 2.2 Preprocess training labels

In [36]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(["Worst Concert Ever", "Did Not Enjoy", "Enjoyed", "Best Concert Ever"])
train_label=le.transform(train_data[['Concert Enjoyment']].to_numpy().ravel())

### 2.3 Standardize and nomalize data

In [37]:
# Assemble all
# Columns not used: ID, Concert goer ID

BDtrain_transformed_mm=minmax_scalar(BDtrain_transformed)
BDtest_transformed_mm=minmax_scalar(BDtest_transformed)

CGAtrain_transformed_ss=standardize_scalar(CGAtrain_transformed)
CGAtest_transformed_ss=standardize_scalar(CGAtest_transformed)

HTtrain_transformed_ss=standardize_scalar(HTtrain_transformed)
HTtest_transformed_ss=standardize_scalar(HTtest_transformed)

### 2.4 Convert country to coordinate

In [38]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
from geopy.extra.rate_limiter import RateLimiter

def dictionaryOfCountries(data, column):
  """Converts country names to corresponding coordinates

  Args:
    data (pandas.DataFrame): DataFrame of the entire dataset
    column (string): Name of column to convert into coordinates

  Returns:
    map (dictionary): Dictionary with (country as string: coordinates as tuple) as key: value

  """
  geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
  map = {}

  unique_countries = data[column].unique()
  for country in unique_countries:
    if isinstance(country, str) and country is not None and country != "Insert Band Country of Origin":
      last_index = country.rfind('(')
      if last_index != -1:
        country = country[0:last_index-1]
      coordinates = geocode(country)
      map[country] = (coordinates.latitude, coordinates.longitude)
    else:
      map[country] = (None, None)

  return map

In [39]:
def country_to_coordinate(country_data: pd.DataFrame, coordinate_dict: dict) -> pd.DataFrame: 
  """ Replace the country name with its coordinate (latitude and longitude).

    Args:
        country_data (pd.DataFrame): country dataframe
        coordinate_dict (dict): dictionary of country's coordinate 
    Returns:
        coordinate_data (pd.DataFrame): coordinate dataframe
  """
  # remove content in the ()
  num_dataset=len(country_data)
  for i in range(num_dataset):
    country_i=str(country_data.iat[i,0])
    last_index = country_i.rfind('(')
    if last_index != -1:
        country_data.iat[i,0] = country_i[0:last_index-1]  
  # convert the country column to coordinate column
  column_name = country_data.columns
  country_data = country_data.rename(columns={column_name[0] : 'Latitude'})
  country_data['Longitude'] = country_data['Latitude']
  # replace country by coordinate
  for country in coordinate_dict.keys():
    country_data['Latitude'] = country_data['Latitude'].replace({country: coordinate_dict[country][0]})
    country_data['Longitude'] = country_data['Longitude'].replace({country: coordinate_dict[country][1]})
  return country_data

In [40]:
dic_Bcountry_coordinates_train=dictionaryOfCountries(train_data,'Band Country of Origin')
band_country_coordinate_train=country_to_coordinate(train_data[['Band Country of Origin']],dic_Bcountry_coordinates_train)

dic_Bcountry_coordinates_test=dictionaryOfCountries(test_data,'Band Country of Origin')
band_country_coordinate_test=country_to_coordinate(test_data[['Band Country of Origin']],dic_Bcountry_coordinates_test)

dic_Ccountry_coordinates_train=dictionaryOfCountries(train_data,'Concert Goer Country of Origin')
cg_country_coordinate_train=country_to_coordinate(train_data[['Concert Goer Country of Origin']],dic_Ccountry_coordinates_train)

dic_Ccountry_coordinates_test=dictionaryOfCountries(test_data,'Concert Goer Country of Origin')
cg_country_coordinate_test=country_to_coordinate(test_data[['Concert Goer Country of Origin']],dic_Ccountry_coordinates_test)

In [41]:
# Replace Nan with mean and convert to numpy array
bc_coordinate_train_transformed=nan_replace_with_mean(band_country_coordinate_train).to_numpy()
bc_coordinate_test_transformed=nan_replace_with_mean(band_country_coordinate_test).to_numpy()
cg_coordinate_train_transformed=nan_replace_with_mean(cg_country_coordinate_train).to_numpy()
cg_coordinate_test_transformed=nan_replace_with_mean(cg_country_coordinate_test).to_numpy()

### 2.5 Band name (Data V3)

In [42]:
# Band Name column
from sklearn import preprocessing
le_2 = preprocessing.LabelEncoder()
le_2.fit(train_data[['Band Name']])

band_name_label_train=le_2.transform(train_data[['Band Name']])
band_name_label_test=le_2.transform(test_data[['Band Name']])

band_name_label_train=band_name_label_train.reshape((len(band_name_label_train), 1))
band_name_label_test=band_name_label_test.reshape((len(band_name_label_test), 1))

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


##### Assemble

In [43]:
# Assemble all
# Columns not used: ID, Concert goer ID
train_set=np.hstack((band_name_label_train, BGtrain_transformed,\
                     bc_coordinate_train_transformed, BDtrain_transformed,\
                     CAtrain_transformed,IVtrain_transformed,RNtrain_transformed,STtrain_transformed,PT1train_transformed,\
                     PT2train_transformed,PT3train_transformed,PT4train_transformed,CGAtrain_transformed,\
                     HTtrain_transformed,cg_coordinate_train_transformed,CItrain_transformed))

test_set=np.hstack((band_name_label_test, BGtest_transformed,\
                    bc_coordinate_test_transformed, BDtest_transformed,\
                    CAtest_transformed,IVtest_transformed,RNtest_transformed,STtest_transformed,PT1test_transformed,\
                    PT2test_transformed,PT3test_transformed,PT4test_transformed,CGAtest_transformed,\
                    HTtest_transformed,cg_coordinate_test_transformed,CItest_transformed))

## 3. Fit, predict and download results (best classifier)

### 3.0 Helper functions

#### 3.0.1 Shuffle

In [44]:
(train_label,train_set)=shuffle(train_label,train_set)

#### 3.0.2 Download

In [45]:
# def download_predicted_label(predicted_label_number):
def download_predicted_label(predicted_label_number, origin_data):
  """ Generate an Excel file with prediction output.

    Args:
        predicted_label_number (List): predicted output
        origin_data (DataFrame): origin input data
  """
  y_pred_label=le.inverse_transform(predicted_label_number)
  y_pred_label = pd.DataFrame(y_pred_label, columns=['Predicted'])
  y_pred_label = pd.concat([origin_data['Id'], y_pred_label], axis=1)
  # To download
  from google.colab import files
  y_pred_label.to_csv('output.csv',encoding='utf-8-sig', index=False) 
  files.download('output.csv')

### 3.1 XGBoost

In [46]:
# calculate class weight
from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=train_label
)

In [47]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# Set the parameters by cross-validation
# Optimal value (V2 data Coodinate): n_estimators=150;
# Optimal value (V3 data BN): n_estimators=50, reg_lambda=32, max_depth=15;
# Optimal value (V4 data BN+BG): n_estimators=100, reg_lambda=12, max_depth=15;
# Optimal value (V5 data CGI): n_estimators=50, reg_lambda=12, max_depth=19, child weight=15;
tuned_parameters = [
  {'n_estimators': [50],
   'max_depth':[15],
   'reg_lambda':[16],
   'min_child_weight':[5]
   },
]
stratified_3_fold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
grid_search = GridSearchCV(
    XGBClassifier(sample_weight=sample_weights), tuned_parameters, scoring='accuracy', cv=stratified_3_fold
)
grid_search.fit(train_set, train_label); # train on the whole training
print('Finished!')

Finished!


In [48]:
print("Best parameter set found on development set:")
print()
print(grid_search.best_params_)

Best parameter set found on development set:

{'max_depth': 15, 'min_child_weight': 5, 'n_estimators': 50, 'reg_lambda': 16}


In [49]:
print("Grid scores on development set:")
print()
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

Grid scores on development set:

0.656 (+/-0.004) for {'max_depth': 15, 'min_child_weight': 5, 'n_estimators': 50, 'reg_lambda': 16}


In [50]:
submit_label=grid_search.predict(test_set)
download_predicted_label(submit_label, test_data)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>